## 1일차 - 강화학습 예제 따라해보기
### 이 코드는 유튜버 Nicholas Renotte 님의 설명을 들으며 MacBook Pro환경에서 실습을 진행한 코드입니다.

- 유튜브 영상 링크 : https://youtu.be/cO5g5qLrLSo

## 실습 필요사항 설치
- tensorflow 2.3.0
- keras
- gym
- keras-rl2
- matplotlib

In [29]:
# !pip install tensorflow==2.3.0
# !pip install keras
# !pip install gym
# !pip install keras-rl2
# !pip install matplotlib

## 실습 시작

In [5]:
import matplotlib.pyplot as plt
import random
import gym

In [6]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [7]:
actions

2

In [8]:
states

4

In [9]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode : {episode} Score : {score}")

Episode : 1 Score : 18.0
Episode : 2 Score : 22.0
Episode : 3 Score : 13.0
Episode : 4 Score : 11.0
Episode : 5 Score : 15.0
Episode : 6 Score : 10.0
Episode : 7 Score : 16.0
Episode : 8 Score : 9.0
Episode : 9 Score : 16.0
Episode : 10 Score : 41.0


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

In [11]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))

    return model

In [12]:
model = build_model(states, actions)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [18]:
model = build_model(states, actions)

In [19]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [20]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)

    return dqn

In [21]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 10:51 - reward: 1.0000

/Users/donghyunjang/PythonHome/RL_Venv/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
93 episodes - episode_reward: 107.011 [10.000, 200.000] - loss: 2.465 - mae: 19.242 - mean_q: 38.959

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 1.0000
50 episodes - episode_reward: 197.780 [179.000, 200.000] - loss: 4.060 - mae: 37.629 - mean_q: 75.897

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.0000
52 episodes - episode_reward: 192.288 [146.000, 200.000] - loss: 3.696 - mae: 39.261 - mean_q: 78.957

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 80s 8ms/step - reward: 1.0000
53 episodes - episode_reward: 190.755 [165.000, 200.000] - loss: 3.585 - mae: 39.136 - mean_q: 78.779

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: 1.0000
done, took 377.815 seconds


In [22]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [23]:
dqn.save_weights('dqn_weights.h5', overwrite=True)

In [24]:
del model
del dqn
del env

In [25]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [26]:
dqn.load_weights('./dqn_weights.h5')

In [27]:
dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
